In [31]:
from sklearn import datasets
import os


In [37]:
# wczytywanie danych

# dir_path = os.path.dirname(os.path.realpath(__file__))
training_set = datasets.load_files("training_set", encoding="utf-8", random_state=420) 
# random_state miesza dane, wartość to seed

In [38]:
# przydatne atrybuty

# training_set.filenames
# training_set.data
# training_set.target_names
# training_set.target
# training_set.target_names[training_set.target[3]]
training_set.data[0]

'niedbal wyspa ogorzały ręka trzymać lejka i odwracać twarz wieźć przez kobieta odpowiadać wesoły zapytanie i przycinek czas męski śmiech swój łączyć ezyt chór cienki piskliwy śmiech dziewczę'

In [39]:
# tokenizacja
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
train_count_vectors = count_vect.fit_transform(training_set.data)
train_count_vectors.shape

(1094, 3583)

In [41]:
# count_vect.vocabulary_.get('koń') # liczba wystąpień słowa w całym zbiorze
# print(train_count_vectors[1])

  (0, 3061)	1
  (0, 494)	1
  (0, 2376)	1
  (0, 2156)	1
  (0, 2487)	1
  (0, 3422)	1
  (0, 1186)	1
  (0, 2378)	1
  (0, 3525)	1
  (0, 1673)	1
  (0, 3180)	1
  (0, 2591)	1
  (0, 714)	1
  (0, 2806)	1
  (0, 349)	1
  (0, 2776)	1
  (0, 1170)	1
  (0, 2738)	1


In [44]:
# tf–idf - “Term Frequency times Inverse Document Frequency”
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
#fit our estimator to the data and transform our count-matrix to a tf-idf representation
train_tfidf = tfidf_transformer.fit_transform(train_count_vectors) 


<1x3583 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [45]:
# Training a classifier
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB().fit(train_tfidf, training_set.target)

In [48]:
# takie sobie testy zdań
test_data = ['Dlaczego z gorącego snu pierwszej młodości obudziła się nie tylko samotna i smutna, ale zarazem obrażona i z niewyschłą dotąd kroplą goryczy w sercu?', 
             'Pod złotawym, a potem już siwiejącym wąsem zawsze purpurowe, zmysłowe jego usta układały się w wyraz lubości, ilekroć zobaczył jakąkolwiek ładną twarzyczkę lub zgrabną kibić niewieścią.',
            'Przed sobą, o kroków kilkanaście, zobaczyła wznoszącą się nad zbożem, rozłożystą i całą w słońcu stojącą gruszę polną; pień, gałęzie i wszystkie liście tego drzewa były złote',
            'Był to wraz z brzegiem rzeki zginający się nieco w półkole sznur siedlisk ludzkich, większych i mniejszych, wychylających ciemne swe profile z większych i mniejszych ogrodów.']
X_new_counts = count_vect.transform(test_data)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = classifier.predict(X_new_tfidf)

for doc, category in zip(test_data, predicted):
     print('%r => %s' % (doc, training_set.target_names[category]))

'Dlaczego z gorącego snu pierwszej młodości obudziła się nie tylko samotna i smutna, ale zarazem obrażona i z niewyschłą dotąd kroplą goryczy w sercu?' => P
'Pod złotawym, a potem już siwiejącym wąsem zawsze purpurowe, zmysłowe jego usta układały się w wyraz lubości, ilekroć zobaczył jakąkolwiek ładną twarzyczkę lub zgrabną kibić niewieścią.' => P
'Przed sobą, o kroków kilkanaście, zobaczyła wznoszącą się nad zbożem, rozłożystą i całą w słońcu stojącą gruszę polną; pień, gałęzie i wszystkie liście tego drzewa były złote' => P
'Był to wraz z brzegiem rzeki zginający się nieco w półkole sznur siedlisk ludzkich, większych i mniejszych, wychylających ciemne swe profile z większych i mniejszych ogrodów.' => P


In [56]:
# Tworzenie Naive Bayes Clf w pipeline i trenowanie

from sklearn.pipeline import Pipeline
import numpy as np

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

training_set = datasets.load_files("training_set", encoding="utf-8", random_state=420) 
clf.fit(training_set.data, training_set.target)

# NotFittedError: CountVectorizer - Vocabulary wasn't fitted. - nwm co jest nie tak
test_set = training_set.data
predicted = clf.predict(test_set)
print(np.mean(predicted == training_set.target))

0.8226691042047533


In [52]:
# import numpy as np

# test_set = training_set.data
# X_new_counts = count_vect.transform(test_set)
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)

# predicted = classifier.predict(X_new_tfidf)
# print(np.mean(predicted == training_set.target))

0.8226691042047533


In [57]:
from sklearn import metrics
print(metrics.classification_report(training_set.target, predicted, target_names=training_set.target_names))

              precision    recall  f1-score   support

           N       0.00      0.00      0.00        34
           O       1.00      0.05      0.10       169
           P       0.82      1.00      0.90       891

   micro avg       0.82      0.82      0.82      1094
   macro avg       0.61      0.35      0.33      1094
weighted avg       0.82      0.82      0.75      1094



c:\users\mike\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [1]:
# # Tworzenie SVM Clf w pipeline i trenowanie
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy as np

svm_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=420,
                          max_iter=5, tol=None)),
])

training_set = datasets.load_files("training_set", encoding="utf-8", random_state=420)
svm_clf.fit(training_set.data, training_set.target)

test_set = training_set.data
predicted = svm_clf.predict(test_set)
print(np.mean(predicted == training_set.target))

0.9986842105263158


c:\users\mike\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [2]:
from sklearn import metrics
print(metrics.classification_report(training_set.target, predicted, target_names=training_set.target_names))

              precision    recall  f1-score   support

           N       1.00      1.00      1.00       144
           O       1.00      0.99      0.99       196
           P       1.00      1.00      1.00      1180

   micro avg       1.00      1.00      1.00      1520
   macro avg       1.00      1.00      1.00      1520
weighted avg       1.00      1.00      1.00      1520



In [2]:
# Grid search dls SVM
from sklearn.model_selection import GridSearchCV

# parameters = {
#     # parametry CountVectorizer
#     """...CountVectorizer(*, input='content', encoding='utf-8', decode_error='strict', 
#     strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\b\w\w+\b', 
#     ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, 
#     dtype=<class 'numpy.int64'>)"""
#     'vect__decode_error': ('strict', 'ignore', 'replace'),
#     'vect__ngram_range': [(1, 1), (1, 2), (2, 2)], # (1, 1), (1, 2), (2, 2), (2, 3), (3, 3) 
#     'vect__max_df': [1.0, 0.90, 0.80, 0.70], # 1.0, 0.95, 0.90, 0.85, 0.80, 0.75, 0.70
#     'vect__min_df': [1, 0.10, 0.20, 0.30] # 0.05, 0.10, 0.15, 0.20, 0.25, 0.30
    
#     # parametry TfidfTransformer
#     """...TfidfTransformer(*, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)"""
#     #można coś jeszcze dodać niby, ale to chyba takie szczegóły, że nie ma sensu
#     'tfidf__use_idf': (True, False),
    
#     # parametry SGDClassifier (SVM)
#     """SGDClassifier(loss='hinge', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
#     tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, 
#     power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, 
#     average=False)"""
#     """'clf__loss': The possible options are ‘hinge’, ‘log’, ‘modified_huber’, ‘squared_hinge’, ‘perceptron’, or a 
#     regression loss: ‘squared_loss’, ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’.
#     Trochę dużo tego """
#     'clf__alpha': (1e-2, 1e-3, 1e-4),
#     'clf__shuffle': (True, False),
#     'clf__class_weight': (None, 'balanced')     
# }


parameters = {
    'vect__decode_error': ('replace',), # ('strict', 'ignore', 'replace')
    'vect__ngram_range': [(1, 2),(2, 2), (2, 3), (3, 3)], # (1, 1), (1, 2), (2, 2), (2, 3), (3, 3) 
    'vect__max_df': [1.0, 0.95, 0.90], # 1.0, 0.95, 0.90, 0.85, 0.80, 0.75, 0.70
    'vect__min_df': [2, 1, 0], # 0.05, 0.10, 0.15, 0.20, 0.25, 0.30 
    'vect__binary': (False,True),
    'tfidf__use_idf': (True,), # zawsze wychodzi True
    'tfidf__norm': ('l1','l2'),
    'tfidf__sublinear_tf': (False, True),
    # można dodać jeszcze kilka parametrów tfidf niby, ale to chyba takie szczegóły, że nie ma sensu
    'clf__alpha': (1e-3, 1e-4, 1e-5),
    'clf__shuffle': (True,), # (True, False)
    'clf__class_weight': (None, 'balanced'), # możemy chcieć None, bo mała część przykładów jest z klasy N
    'clf__penalty': ('l1','l2'), #('l1','l2')
#     'clf__loss': ('hinge', 'log', 'modified_huber')
#     ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 
#       'epsilon_insensitive', 'squared_epsilon_insensitive')
}

grid_search_clf = GridSearchCV(svm_clf, parameters, cv=5, n_jobs=3) # !!! n_jobs - LICZBA UŻYWANYCH RDZENI !!!

In [3]:
grid_search_clf = grid_search_clf.fit(training_set.data, training_set.target)
# grid_search_clf.get_params().keys()

c:\users\mike\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
c:\users\mike\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [4]:
print(grid_search_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, grid_search_clf.best_params_[param_name]))
    
"""
0.8546617915904936
clf__alpha: 0.001
clf__class_weight: 'balanced'
clf__penalty: 'l2'
clf__shuffle: True
tfidf__use_idf: True
vect__binary: True
vect__decode_error: 'strict'
vect__max_df: 1.0
vect__min_df: 1
vect__ngram_range: (1, 2)


0.8208409506398537
clf__alpha: 0.0001
clf__class_weight: None
clf__penalty: 'l2'
clf__shuffle: True
tfidf__norm: 'l2'
tfidf__sublinear_tf: False
tfidf__use_idf: True
vect__binary: False
vect__decode_error: 'replace'
vect__max_df: 1.0
vect__min_df: 1
vect__ngram_range: (2, 2) # nie było opcji (1, 2)
"""

0.8322368421052632
clf__alpha: 0.0001
clf__class_weight: 'balanced'
clf__loss: 'hinge'
clf__penalty: 'l2'
clf__shuffle: True
tfidf__norm: 'l2'
tfidf__sublinear_tf: False
tfidf__use_idf: True
vect__binary: True
vect__decode_error: 'replace'
vect__max_df: 1.0
vect__min_df: 1
vect__ngram_range: (1, 2)


"\n0.8546617915904936\nclf__alpha: 0.001\nclf__class_weight: 'balanced'\nclf__penalty: 'l2'\nclf__shuffle: True\ntfidf__use_idf: True\nvect__binary: True\nvect__decode_error: 'strict'\nvect__max_df: 1.0\nvect__min_df: 1\nvect__ngram_range: (1, 2)\n\n\n0.8208409506398537\nclf__alpha: 0.0001\nclf__class_weight: None\nclf__penalty: 'l2'\nclf__shuffle: True\ntfidf__norm: 'l2'\ntfidf__sublinear_tf: False\ntfidf__use_idf: True\nvect__binary: False\nvect__decode_error: 'replace'\nvect__max_df: 1.0\nvect__min_df: 1\nvect__ngram_range: (2, 2) # nie było opcji (1, 2)\n"

In [21]:
print(grid_search_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, grid_search_clf.best_params_[param_name]))
    
"""
0.8208409506398537
clf__alpha: 0.0001
clf__class_weight: None
clf__penalty: 'l2'
clf__shuffle: True
tfidf__norm: 'l2'
tfidf__sublinear_tf: False
tfidf__use_idf: True
vect__binary: False
vect__decode_error: 'replace'
vect__max_df: 1.0
vect__min_df: 1
vect__ngram_range: (2, 2)
"""

0.8208409506398537
clf__alpha: 0.0001
clf__class_weight: None
clf__penalty: 'l2'
clf__shuffle: True
tfidf__norm: 'l2'
tfidf__sublinear_tf: False
tfidf__use_idf: True
vect__binary: False
vect__decode_error: 'replace'
vect__max_df: 1.0
vect__min_df: 1
vect__ngram_range: (2, 2)


In [11]:
# 'Był to wraz z brzegiem rzeki zginający się nieco w półkole sznur siedlisk ludzkich, większych i mniejszych, wychylających ciemne swe profile z większych i mniejszych ogrodów.'
tag = grid_search_clf.predict(['drzewo ogórd słońce las kwiat'])[0]
print(training_set.target_names[tag])

P


In [5]:
"""
0.8322368421052632
clf__alpha: 0.0001
clf__class_weight: 'balanced'
clf__loss: 'hinge'
clf__penalty: 'l2'
clf__shuffle: True
tfidf__norm: 'l2'
tfidf__sublinear_tf: False
tfidf__use_idf: True
vect__binary: True
vect__decode_error: 'replace'
vect__max_df: 1.0
vect__min_df: 1
vect__ngram_range: (1, 2)
"""
# # Tworzenie SVM Clf w pipeline i trenowanie
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy as np

svm_clf = Pipeline([
    ('vect', CountVectorizer(binary=True, decode_error='replace', max_df=1.0, min_df=1, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(norm='l2', sublinear_tf=False, use_idf=True)),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, class_weight='balanced', random_state=420,
                          shuffle=True)),
])

training_set = datasets.load_files("training_set", encoding="utf-8", random_state=420)
svm_clf.fit(training_set.data, training_set.target)

test_set = training_set.data
predicted = svm_clf.predict(test_set)
print(np.mean(predicted == training_set.target))

from sklearn import metrics
print(metrics.classification_report(training_set.target, predicted, target_names=training_set.target_names))

0.9986842105263158
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       144
           O       0.99      1.00      0.99       196
           P       1.00      1.00      1.00      1180

   micro avg       1.00      1.00      1.00      1520
   macro avg       1.00      1.00      1.00      1520
weighted avg       1.00      1.00      1.00      1520



c:\users\mike\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [7]:
import pickle

with open('svm_clf.pickle', 'wb') as f:
    pickle.dump(svm_clf, f)

In [12]:
# del svm_clf

with open('svm_clf.pickle', 'rb') as f:
    loaded_svm_clf = pickle.load(f)
    
tag = loaded_svm_clf.predict(['drzewo ogórd słońce las kwiat'])[0]
print(training_set.target_names[tag])

N


In [15]:
# training_set.target_names
target_names = ['N', 'O', 'P']

test_data = ['Dlaczego z gorącego snu pierwszej młodości obudziła się nie tylko samotna i smutna, ale zarazem obrażona i z niewyschłą dotąd kroplą goryczy w sercu?', 
             'Pod złotawym, a potem już siwiejącym wąsem zawsze purpurowe, zmysłowe jego usta układały się w wyraz lubości, ilekroć zobaczył jakąkolwiek ładną twarzyczkę lub zgrabną kibić niewieścią.',
            'Przed sobą, o kroków kilkanaście, zobaczyła wznoszącą się nad zbożem, rozłożystą i całą w słońcu stojącą gruszę polną; pień, gałęzie i wszystkie liście tego drzewa były złote',
            'Był to wraz z brzegiem rzeki zginający się nieco w półkole sznur siedlisk ludzkich, większych i mniejszych, wychylających ciemne swe profile z większych i mniejszych ogrodów.']

predicted = loaded_svm_clf.predict(test_data)

for doc, category in zip(test_data, predicted):
     print('%r => %s' % (doc, target_names[category]))

'Dlaczego z gorącego snu pierwszej młodości obudziła się nie tylko samotna i smutna, ale zarazem obrażona i z niewyschłą dotąd kroplą goryczy w sercu?' => P
'Pod złotawym, a potem już siwiejącym wąsem zawsze purpurowe, zmysłowe jego usta układały się w wyraz lubości, ilekroć zobaczył jakąkolwiek ładną twarzyczkę lub zgrabną kibić niewieścią.' => O
'Przed sobą, o kroków kilkanaście, zobaczyła wznoszącą się nad zbożem, rozłożystą i całą w słońcu stojącą gruszę polną; pień, gałęzie i wszystkie liście tego drzewa były złote' => P
'Był to wraz z brzegiem rzeki zginający się nieco w półkole sznur siedlisk ludzkich, większych i mniejszych, wychylających ciemne swe profile z większych i mniejszych ogrodów.' => P


In [16]:
#kod do skryptu całego programu
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy as np
import pickle


with open('svm_clf.pickle', 'rb') as f:
    svm_clf = pickle.load(f)
    
    target_names = ['N', 'O', 'P']

    # klasyfikacja 1
    test_data = ['Dlaczego z gorącego snu pierwszej młodości obudziła się nie tylko samotna i smutna, ale zarazem obrażona i z niewyschłą dotąd kroplą goryczy w sercu?', 
                 'Pod złotawym, a potem już siwiejącym wąsem zawsze purpurowe, zmysłowe jego usta układały się w wyraz lubości, ilekroć zobaczył jakąkolwiek ładną twarzyczkę lub zgrabną kibić niewieścią.',
                'Przed sobą, o kroków kilkanaście, zobaczyła wznoszącą się nad zbożem, rozłożystą i całą w słońcu stojącą gruszę polną; pień, gałęzie i wszystkie liście tego drzewa były złote',
                'Był to wraz z brzegiem rzeki zginający się nieco w półkole sznur siedlisk ludzkich, większych i mniejszych, wychylających ciemne swe profile z większych i mniejszych ogrodów.']

    predicted = svm_clf.predict(test_data) # zwraca id klas; target_names[id] - nazwa klasy

    for doc, category in zip(test_data, predicted):
         print('%r => %s' % (doc, target_names[category]))

    # klasyfikacja 2
    tag = svm_clf.predict(['drzewo ogórd słońce las kwiat'])[0]
    print('drzewo ogórd słońce las kwiat => '+target_names[tag])

'Dlaczego z gorącego snu pierwszej młodości obudziła się nie tylko samotna i smutna, ale zarazem obrażona i z niewyschłą dotąd kroplą goryczy w sercu?' => P
'Pod złotawym, a potem już siwiejącym wąsem zawsze purpurowe, zmysłowe jego usta układały się w wyraz lubości, ilekroć zobaczył jakąkolwiek ładną twarzyczkę lub zgrabną kibić niewieścią.' => O
'Przed sobą, o kroków kilkanaście, zobaczyła wznoszącą się nad zbożem, rozłożystą i całą w słońcu stojącą gruszę polną; pień, gałęzie i wszystkie liście tego drzewa były złote' => P
'Był to wraz z brzegiem rzeki zginający się nieco w półkole sznur siedlisk ludzkich, większych i mniejszych, wychylających ciemne swe profile z większych i mniejszych ogrodów.' => P
N
